# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [3]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [4]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [5]:
#clean profile data

print(profile.isnull().any())
print(profile.gender.unique())

values = {'gender': 'M', 'income': round(profile.income.mean())}

profile = profile.fillna(value=values)
profile.head()

gender               True
age                 False
id                  False
became_member_on    False
income               True
dtype: bool
[None 'F' 'M' 'O']


,gender,age,id,became_member_on,income
0,M,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,65405.0
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,M,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,65405.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,M,118,a03223e636434f42ac4c3df47e8bac43,20170804,65405.0


- 'offer_sent' - 1 - yes, 0 - no
- 'offer_id' - offer_id -yes, else - 0
- 'offer_viewed' 1 - yes, 0 - no
- 'no_txn' - count
- 'offer_used' - 1- yes, 0 - no

In [13]:
trasncript_sorted = transcript.sort_values('person',ignore_index=True)
list(trasncript_sorted.loc[1,'value'].values())[0]

8.57

In [14]:
transcript_sorted['person'].value_counts().head()


94de646f7b6041228ca7dec82adb97d2    51
8dbfa485249f409aa223a2130f40634a    49
5e60c6aa3b834e44b822ea43a3efea26    48
d0a80415b84c4df4908b8403b19765e3    48
79d9d4f86aca4bed9290350fb43817c2    48
Name: person, dtype: int64

In [15]:
temp_df = transcript_sorted[transcript_sorted['person'] == '94de646f7b6041228ca7dec82adb97d2']

temp_df.sort_values('time')

,person,event,value,time
178566,94de646f7b6041228ca7dec82adb97d2,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
178544,94de646f7b6041228ca7dec82adb97d2,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},6
178591,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 7.41},30
178589,94de646f7b6041228ca7dec82adb97d2,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,30
178545,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 1.47},102
178560,94de646f7b6041228ca7dec82adb97d2,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},168
178547,94de646f7b6041228ca7dec82adb97d2,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},186
178592,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 2.62},192
178590,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 0.59},204
178556,94de646f7b6041228ca7dec82adb97d2,transaction,{'amount': 2.2800000000000002},246


In [20]:
def get_rows_for_person(transaction, person, data):
    """
    This function is for returning rows of a perticular person
    the number of rows are the number of offers the person completed/received
    
    input - complete transcript, person of interest
    
    output - the records for the person added to data
    
    """
    
    temp_df = transaction[transaction['person'] == person]
    
    # get the txn amount or offer id from each row, and drop original value column
    txn_type = []
    for val in temp_df.value:
        txn_type.append(list(val.values())[0])

    temp_df['txn_amt_offer'] = txn_type
    temp_df = temp_df.drop('value',axis=1)

    
    # get no. of txns and the amount spent
    no_txn = len(temp_df[temp_df['event'] == 'transaction'])
    total_amount = round(temp_df[temp_df['event'] == 'transaction'].txn_amt_offer.sum())
    
    
    # drop all rows with transactions as event
    temp_df = temp_df[temp_df.event != 'transaction']
    
    # check details of each offer, and add a row to output data
    for offer in temp_df.txn_amt_offer.unique():
        offer_temp_df = temp_df[temp_df.txn_amt_offer == offer]
        received = len(offer_temp_df[offer_temp_df['event'] == 'offer received'])
        viewed = len(offer_temp_df[offer_temp_df['event'] == 'offer viewed'])
        completed = len(offer_temp_df[offer_temp_df['event'] == 'offer completed'])
#         print(person, offer, received, viewed, no_txn, total_amount, completed)
        
        data = data.append({'person' : person,
                            'offer_id' : offer,
                            'offer_received' : received, 
                            'offer_viewed' : viewed,
                            'no_txn' : no_txn,
                            'amount_spent' : total_amount,
                            'offer_completed' : completed}, ignore_index=True)    
    
    
    return data




In [21]:
data = pd.DataFrame(columns=['person','offer_id','offer_received','offer_viewed','no_txn','amount_spent','offer_completed'])

# transcript_sorted.person.unique():


# for person in transcript_sorted.person.unique():
#     data = get_rows_for_person(transcript_sorted, person, data)


    
# data

In [22]:
#save the data to file
# commenting to prevent accidental overwrite
#data.to_csv('data/data.csv', index=False, header=True)

In [43]:
data = pd.read_csv('data/data.csv')
data

,person,offer_id,offer_received,offer_viewed,no_txn,amount_spent,offer_completed
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1,8,128,0
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1,8,128,1
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1,8,128,1
3,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,0,8,128,1
4,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1,8,128,0
...,...,...,...,...,...,...,...
63283,fffad4f4828548d1b5583907f2e9906b,5a8bc65990b245e5a138643cd4eb9837,1,1,12,89,0
63284,ffff82501cea40309d5fdd7edcca4a07,2906b810c7d4411798c6938adc9daaa5,3,3,15,226,3
63285,ffff82501cea40309d5fdd7edcca4a07,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,15,226,1
63286,ffff82501cea40309d5fdd7edcca4a07,fafdcd668e3743c1bb461111dcafc2a4,1,1,15,226,1


In [44]:
# set output = 0 or 1.
# 1 means to keep sending offers
# 0 is to stop sending offers
# if a person has viewed and used an offer, or if he has not viewed and not used.. V - C = 0 keep sending more offers
# if a person has viewed offers but not used all of them. V-C > 0 & C > 0 .. keep sending offers
# if a person has seen offers but never used them. V-C > 0 & C = 0.. Do not send more offers
# if a person has used more offers than the offers viewd. V-C < 0 .. Do not send more offers

output = data.apply(lambda 
                    row: 
                    1 if 
                    (row.offer_viewed-row.offer_completed==0) or 
                    (row.offer_viewed-row.offer_completed > 0 and row.offer_completed > 0) 
                    else 0, axis=1)

data['output'] = output
data.head()

,person,offer_id,offer_received,offer_viewed,no_txn,amount_spent,offer_completed,output
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1,8,128,0,0
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1,8,128,1,1
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1,8,128,1,1
3,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,0,8,128,1,0
4,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1,8,128,0,0


In [45]:
data_mid = pd.merge(data, profile, left_on='person', right_on='id')
drop_columns = ['id','became_member_on']
data_mid = data_mid.drop(drop_columns,axis=1)
data_mid.head(10)

,person,offer_id,offer_received,offer_viewed,no_txn,amount_spent,offer_completed,output,gender,age,income
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1,8,128,0,0,M,33,72000.0
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1,8,128,1,1,M,33,72000.0
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1,8,128,1,1,M,33,72000.0
3,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,0,8,128,1,0,M,33,72000.0
4,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1,8,128,0,0,M,33,72000.0
5,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,2,2,3,4,0,0,M,118,65405.0
6,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,5,79,1,1,O,40,57000.0
7,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,5,79,1,1,O,40,57000.0
8,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,1,1,5,79,0,0,O,40,57000.0
9,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,1,1,5,79,0,0,O,40,57000.0


In [46]:
final_data = pd.merge(data_mid, portfolio, left_on='offer_id', right_on='id', how='left')
drop_columns = ['id','channels']
final_data = final_data.drop(drop_columns,axis=1)
final_data.head()

,person,offer_id,offer_received,offer_viewed,no_txn,amount_spent,offer_completed,output,gender,age,income,reward,difficulty,duration,offer_type
0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,1,1,8,128,0,0,M,33,72000.0,0,0,3,informational
1,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,1,1,8,128,1,1,M,33,72000.0,5,5,5,bogo
2,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,1,1,8,128,1,1,M,33,72000.0,2,10,10,discount
3,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,1,0,8,128,1,0,M,33,72000.0,2,10,7,discount
4,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,1,1,8,128,0,0,M,33,72000.0,0,0,4,informational


# Final cleansing

Now that we have all required data inone data frame. 
- Offer_id and person can be dropped.
- gender and offer_type needs to be one-hot encoded
- income canbe normalized


In [47]:
drop_columns = ['person','offer_id','income']
encode_columns = ['gender','offer_type']

# normalize income
normalized_income=(final_data.income-final_data.income.min())/(final_data.income.max()-final_data.income.min())
final_data['normalized_income'] = normalized_income

# encode columns and drop extra columns
final_data = pd.get_dummies(final_data,columns=encode_columns)
final_data = final_data.drop(drop_columns,axis=1)

final_data.head(10)

,offer_received,offer_viewed,no_txn,amount_spent,offer_completed,output,age,reward,difficulty,duration,normalized_income,gender_F,gender_M,gender_O,offer_type_bogo,offer_type_discount,offer_type_informational
0,1,1,8,128,0,0,33,0,0,3,0.466667,0,1,0,0,0,1
1,1,1,8,128,1,1,33,5,5,5,0.466667,0,1,0,1,0,0
2,1,1,8,128,1,1,33,2,10,10,0.466667,0,1,0,0,1,0
3,1,0,8,128,1,0,33,2,10,7,0.466667,0,1,0,0,1,0
4,1,1,8,128,0,0,33,0,0,4,0.466667,0,1,0,0,0,1
5,2,2,3,4,0,0,118,5,5,5,0.393389,0,1,0,1,0,0
6,1,1,5,79,1,1,40,3,7,7,0.300000,0,0,1,0,1,0
7,1,1,5,79,1,1,40,5,20,10,0.300000,0,0,1,0,1,0
8,1,1,5,79,0,0,40,0,0,3,0.300000,0,0,1,0,0,1
9,1,1,5,79,0,0,40,0,0,4,0.300000,0,0,1,0,0,1


# Train and test the model

- keep sending offers if the output is 1
- do not send offers if the output is 0

In [48]:
y = final_data['output']
X = final_data.drop(['output'],axis=1)

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                          n_estimators=200)

model.fit(X_train, y_train)

pred = model.predict(X_test)

accuracy = accuracy_score(y_test, pred)

print('accuracy : ', accuracy)

accuracy :  0.9999521210380159


In [55]:
fbeta_score(y_test, pred, beta = 0.5)

0.99992735725701